# Section 8：1人分の月次データを集計しよう

## Excelファイルの読み込み

In [ ]:
import openpyxl

In [ ]:
wb_members = openpyxl.load_workbook('従業員集約.xlsx', data_only=True)

In [ ]:
wb_members.sheetnames

In [ ]:
names = wb_members.sheetnames
names

In [ ]:
name = names[0]
name

## 2月分のデータを取得しよう

In [ ]:
import pandas as pd

In [ ]:
# nameには佐野和也が入っている
df = pd.read_excel('従業員集約.xlsx', sheet_name=name)

In [ ]:
# 先頭5行
df.head()

In [ ]:
import datetime

In [ ]:
year = 2022
month = 2
start_date = datetime.date(year, month, 1)
start_date

In [ ]:
# 3月1日から1日差し引いた日付を取得
end_date = (datetime.date(year, month+1, 1)
            - datetime.timedelta(days=1))
end_date

In [ ]:
df['日付'].dt.date >= start_date

In [ ]:
df['日付'].dt.date <= end_date

In [ ]:
(df['日付'].dt.date >= start_date) & (df['日付'].dt.date <= end_date)

In [ ]:
# df[先程の条件式]
df_filtered = df[(df['日付'].dt.date >= start_date) 
                 & (df['日付'].dt.date <= end_date)]
df_filtered

## 合計作業時間を取得しよう

In [ ]:
# 引数に指定したvalは各値になります
def to_minute(val):
    # 時間×60+分
    return val.hour*60 + val.minute

In [ ]:
df_filtered['作業時間'].fillna(datetime.time())

In [ ]:
df_filtered['作業時間'].fillna(datetime.time()).apply(to_minute)

In [ ]:
df_filtered_fillna = df_filtered['作業時間'].fillna(datetime.time())
working_hours = df_filtered_fillna.apply(to_minute)
total_working_hour = working_hours.sum()/60
total_working_hour

In [ ]:
datum = {
    '氏名': name,
    '合計作業時間': total_working_hour
}
datum

## まとめ

In [ ]:
import openpyxl
import pandas as pd
import datetime

def to_minute(val):
    # 時間×60+分
    return val.hour*60 + val.minute

wb_members = openpyxl.load_workbook(
    '従業員集約.xlsx', data_only=True)
names = wb_members.sheetnames

df = pd.read_excel('従業員集約.xlsx', sheet_name=name)

year = 2022
month = 2
start_date = datetime.date(year, month, 1)
end_date = (datetime.date(year, month+1, 1)
            - datetime.timedelta(days=1))

# df[条件式]で指定期間内のデータを抽出
df_filtered = df[(df['日付'].dt.date >= start_date) 
                 & (df['日付'].dt.date <= end_date)]

df_filtered_fillna = df_filtered['作業時間'].fillna(
    datetime.time())
working_hours = df_filtered_fillna.apply(to_minute)
total_working_hour = working_hours.sum()/60

datum = {
    '氏名': name,
    '合計作業時間': total_working_hour
}
datum

# Section 9：全員分の月次データを集計しよう

In [ ]:
import openpyxl
import pandas as pd
import datetime

def to_minute(val):
    # 時間×60+分
    return val.hour*60 + val.minute

wb_members = openpyxl.load_workbook('従業員集約.xlsx', data_only=True)
names = wb_members.sheetnames

# 追記①
data = []
# for文に修正
for name in names:
    df = pd.read_excel('従業員集約.xlsx', sheet_name=name)

    year = 2022
    month = 2
    start_date = datetime.date(year, month, 1)
    end_date = (datetime.date(year, month+1, 1)
                - datetime.timedelta(days=1))

    # df[条件式]で指定期間内のデータを抽出
    df_filtered = df[(df['日付'].dt.date >= start_date) 
                     & (df['日付'].dt.date <= end_date)]

    df_filtered_fillna = df_filtered['作業時間'].fillna(datetime.time())
    working_hours = df_filtered_fillna.apply(to_minute)
    total_working_hour = working_hours.sum()/60

    datum = {
        '氏名': name,
        '合計作業時間': total_working_hour
    }
    # 追記②
    data.append(datum)

In [ ]:
data

In [ ]:
df_summary = pd.DataFrame(data)
df_summary

# Section10：集計データをExcelファイルへ出力しよう

## 集計用Excelファイルを新規作成する

In [ ]:
wb_summary = openpyxl.Workbook()
# 集計.xlsxというファイル名でWorkbookを保存
wb_summary.save('集計.xlsx')

In [ ]:
# 集計.xlsxファイルを読み込む
wb_summary = openpyxl.load_workbook('集計.xlsx', data_only=True)

In [ ]:
# 作成した「2022年2月分」シートを変数ws_summaryに代入
ws_summary = wb_summary.create_sheet(title=f'{year}年{month}月分')

In [ ]:
# シートが作成されたかを確認
wb_summary.sheetnames

In [ ]:
wb_summary.remove(wb_summary['Sheet'])
# 削除を反映するために保存
wb_summary.save('集計.xlsx')

## 集計データをExcelへ書き込む

In [ ]:
# カラムの用意
ws_summary['A1'] = '氏名'
ws_summary['B1'] = '合計作業時間'

ws_summary.column_dimensions['A'].width = 15
ws_summary.column_dimensions['B'].width = 15

In [ ]:
for index, row in df_summary.iterrows():
    ws_summary.cell(row=index+2, column=1).value = row['氏名']
    ws_summary.cell(row=index+2, column=2).value = row['合計作業時間']

In [ ]:
wb_summary.save('集計.xlsx')

# Section 11：Pythonファイルに出力処理をまとめる

In [ ]:
text = '202203'
text[:4]

In [ ]:
text[4:]

In [ ]:
year = int(text[:4])
year

In [ ]:
month = int(text[4:])
month

### 【注意】
以下のプログラムは、ターミナルから実行する**Pythonファイル**として調整しています。<br>
**ノートブック上ではエラーが発生**し、正常に実行できません。

In [ ]:
import openpyxl
import pandas as pd
import datetime
# 追加
import sys
args = sys.argv
year = int(args[1][:4])
month = int(args[1][4:])

def to_minute(val):
    return val.hour*60 + val.minute

wb_members = openpyxl.load_workbook(
    '従業員集約.xlsx', data_only=True)
names = wb_members.sheetnames

data = []
for name in names:
    df = pd.read_excel('従業員集約.xlsx', sheet_name=name)
    start_date = datetime.date(year, month, 1)
    end_date = (datetime.date(year, month+1, 1)
                - datetime.timedelta(days=1))

    df_filtered = df[(df['日付'].dt.date >= start_date) 
                     & (df['日付'].dt.date <= end_date)]
    df_filtered_fillna = df_filtered['作業時間'].fillna(
        datetime.time())
    working_hours = df_filtered_fillna.apply(to_minute)
    total_working_hour = working_hours.sum()/60

    datum = {
        '氏名': name,
        '合計作業時間': total_working_hour
    }
    data.append(datum)

df_summary = pd.DataFrame(data)

wb_summary = openpyxl.load_workbook('集計.xlsx', data_only=True)
ws_summary = wb_summary.create_sheet(title=f'{year}年{month}月分')

ws_summary['A1'] = '氏名'
ws_summary['B1'] = '合計作業時間'

ws_summary.column_dimensions['A'].width = 15
ws_summary.column_dimensions['B'].width = 15

for index, row in df_summary.iterrows():
    ws_summary.cell(row=index+2, column=1).value = row['氏名']
    ws_summary.cell(row=index+2, column=2).value = row['合計作業時間']

wb_summary.save('集計.xlsx')
print(f'{year}年{month}月分の集計完了')